In [2]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt
import optuna

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps")
device

device(type='mps')

In [6]:
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
path = "./dataset"

dataset = datasets.ImageFolder(root=path, transform=image_transforms)
len(dataset)

2300

In [14]:
class_names=dataset.classes
class_names

['F_Breakage', 'F_Crushed', 'F_Normal', 'R_Breakage', 'R_Crushed', 'R_Normal']

In [28]:
num_classes=len(dataset.classes)
num_classes

6

In [20]:
train_size=int(len(dataset)*0.75)
val_size=len(dataset)-train_size
print(train_size,val_size)

1725 575


In [22]:
from torch.utils.data import random_split

train_dataset,val_dataset=random_split(dataset,[train_size,val_size])

In [24]:
train_loader=DataLoader(train_dataset,shuffle=True,batch_size=64)
val_loader=DataLoader(val_dataset,shuffle=True,batch_size=64)

In [26]:
for images,labels in train_loader:
    print(images.shape)
    print(labels.shape)
    break

torch.Size([64, 3, 224, 224])
torch.Size([64])


In [40]:
#model training and hyperparameter tuning


class carclassifierResnet50(nn.Module):
    def __init__(self,num_classes,dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')
        for params in self.model.parameters():
            params.requires_grad=False
        for params in self.model.layer4.parameters():
            params.requires_grad=True
        self.model.fc=nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features,num_classes))

    def forward(self,x):
            x=self.model(x)
            return x  

In [48]:
def objective(trail):
    dropout_rate=trail.suggest_float("dropout_rate",0.2,0.7)
    lr=trail.suggest_float("lr",1e-5,1e-2,log=True)

    model=carclassifierResnet50(num_classes=num_classes,dropout_rate=dropout_rate).to(device)

    optimizer=optim.Adam(filter(lambda p:p.requires_grad,model.parameters()),lr=lr)
    criterion=nn.CrossEntropyLoss()


    #training loop
    epochs=2
    start=time.time()
    for epoch in range(epochs):
        model.train()
        training_loss=0
        for batch,(images,labels) in enumerate(train_loader):
            images,labels=images.to(device),labels.to(device)
            optimizer.zero_grad()
            output=model(images)
            loss=criterion(output,labels)
            loss.backward()
            optimizer.step()
            training_loss+=loss.item()*images.size(0)
        epoch_loss=training_loss/len(train_loader.dataset)

        #validation
        model.eval()
        correct=0
        total=0
        with torch.no_grad():
            for images,labels in val_loader:
                images,labels=images.to(device),labels.to(device)
                output=model(images)
                _,predicted=torch.max(output.data,1)
                total+=labels.size(0)
                correct+=(labels==predicted).sum().item()

        accuracy=(correct*100)/total
        trail.report(accuracy,epoch)
        #handle pruning(if applicable)
        if trail.should_prune():
            raise optuna.exceptions.TrialPruned()

    end=time.time()
    print(f"execution time {end-start} seconds")
    return  accuracy
            
#create study and optimize

study=optuna.create_study(direction="maximize")
study.optimize(objective,n_trials=20)

[I 2025-04-16 19:56:59,320] A new study created in memory with name: no-name-fb5c68c5-71fe-4f4b-81cf-adc82c942807
[I 2025-04-16 19:59:13,048] Trial 0 finished with value: 69.91304347826087 and parameters: {'dropout_rate': 0.24449142096755172, 'lr': 0.0003310013721784448}. Best is trial 0 with value: 69.91304347826087.


execution time 133.06328511238098 seconds


[I 2025-04-16 20:01:21,896] Trial 1 finished with value: 75.30434782608695 and parameters: {'dropout_rate': 0.36137896746955295, 'lr': 0.0015735613001326037}. Best is trial 1 with value: 75.30434782608695.


execution time 128.3889570236206 seconds


[I 2025-04-16 20:03:33,377] Trial 2 finished with value: 73.56521739130434 and parameters: {'dropout_rate': 0.5760198923231112, 'lr': 0.00535057099354885}. Best is trial 1 with value: 75.30434782608695.


...

[I 2025-04-16 20:13:11,823] Trial 7 pruned. 
[I 2025-04-16 20:14:28,099] Trial 8 pruned. 
[I 2025-04-16 20:15:42,119] Trial 9 pruned. 
[I 2025-04-16 20:16:57,178] Trial 10 pruned. 
[I 2025-04-16 20:19:29,939] Trial 11 pruned. 
[I 2025-04-16 20:21:59,537] Trial 12 finished with value: 75.82608695652173 and parameters: {'dropout_rate': 0.6959276846850988, 'lr': 0.0016719975945587572}. Best is trial 3 with value: 77.3913043478261.


execution time 149.12617588043213 seconds


[I 2025-04-16 20:23:13,919] Trial 13 pruned. 
[I 2025-04-16 20:24:28,640] Trial 14 pruned. 
[I 2025-04-16 20:26:58,869] Trial 15 finished with value: 76.34782608695652 and parameters: {'dropout_rate': 0.6432794161137829, 'lr': 0.000247746446157692}. Best is trial 3 with value: 77.3913043478261.


execution time 149.7555446624756 seconds


[I 2025-04-16 20:28:13,567] Trial 16 pruned. 
[I 2025-04-16 20:29:29,287] Trial 17 pruned. 
[I 2025-04-16 20:30:44,124] Trial 18 pruned. 
[I 2025-04-16 20:31:59,434] Trial 19 pruned. 


If trial.should_prune(): raise TrialPruned()
This tells Optuna to check if this trial should be stopped early.

Optuna compares this trial’s progress to others (based on a pruning strategy like MedianPruner, SuccessiveHalvingPruner, etc.).

If the current trial is performing worse than expected at this stage, Optuna prunes it (stops it early)

In [51]:
study.best_params

{'dropout_rate': 0.6369420038515443, 'lr': 0.001329007900191198}